# Week 3: Least Squares Fitting (cont.)

## Goals
- Nonlinear fitting
- Computing $r^2$ values

## Nonlinear fitting

We will take the example we looked at in class and find a parabola of best fit. 

Thus, we want to find coefficients for 
$$
    y = b_0 + b_1x_1 + b_2x_2. 
$$

The data for this example is found in `data/nonlinear_ex.csv`.

In [ ]:
import pandas as pd 

df = pd.read_csv("data/nonlinear_ex.csv")
print(df)

Alternatively, you can run the following code to get the same data. To turn it "on" remove all of the `#` symbols. 

In [ ]:
# df = pd.DataFrame({
#     "x_i" : [2.27, 5.06, 1.45, 5.89, 0.48, -0.22, 1.44, -1.77, 2.45, -1.54, 7.55, 1.76, 5.16, 3.26, 3.23, 0.85],
#     "y_i" : [2.5, -16.13, 4.23, -22.46, 1.37, 0.86, 11.85, -14.71, 9.42, -14.07, -55.62, 4.45, -19.56, -2.79, 5.2, 8.09],
# })

Let's plot our data and verify it is what we expect. 

In [ ]:
import matplotlib.pyplot as plt 

fig, ax = plt.subplots()
ax.scatter(df["x_i"], df["y_i"])
ax.grid()

Good. It appears that a parabola might be good enough to describe the general trend of the data. 

Let's try to fit the parabola:
$$
    y = b_0 + b_1x + b_2x^2
$$

From the lecture, we want to *instead* look for the plane of best fit: 
$$
    y = b_0 + b_1x + b_2z,
$$

where $z=x^2$.

This means we need a new columns for $z$, and we know how to get it from $x_i$.

We build a new data frame.

In [ ]:
df2 = pd.DataFrame({
    "x_i" : df["x_i"],
    "x_i^2" : [x**2 for x in df["x_i"]],
    "y_i" : df["y_i"],
})
print(df2)

Now we compute the plane of best fit.

First we build the matrix $X$

In [ ]:
import numpy as np 

X = np.array([
    [1]*len(df2),
    df2["x_i"],
    df2["x_i^2"],
]).T
print(X)

And then we build the matrix (or column vector) $Y$.

In [ ]:
Y = np.array([df2["y_i"]]).T
print(Y)

Recall the formula for the $b_i$ values: 
$$
    X^{\mathrm{t}}XB = X^{\mathrm{t}} Y.
$$

In [ ]:
B = np.linalg.inv(X.T @ X) @ X.T @ Y 
print(B)

Therefore the *plane* of best for the data frame `df2` is 
$$
    y = 1.30 + 6.03x - 1.81z.
$$

Since $z=x^2$, the *parabola* of best fit for the data frame `df` is 
$$
    y = 1.30 + 6.03x - 1.81x^2. 
$$

**Note.** We declared the variables $x$ and $z$ to be independent---that is how we got the plane of best fit; however this is not the case, and that is fine. The data in `df2` have a higher redundancy than the data in `df`.

Now let's plot the scatter plot together with our parabola.

First, we get all of our data in order to plot the parabola.

In [ ]:
xs = np.linspace(-2, 8, 100)
ys = B[0,0] + B[1,0]*xs + B[2,0]*xs**2

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df["x_i"], df["y_i"])
ax.plot(xs, ys, c="orange")
ax.grid()

Not bad! 😃

## Computing $r^2$ values